# Part I. ETL Pipeline for Pre-Processing the Files

#### Import Python packages 

In [1]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### Creating list of filepaths to process original event csv data files

In [2]:
# checking your current working directory
print('current working directory: ',os.getcwd())

# Get your current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath): 
    
    # join the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*')) 
    
print('\nProvided data files:')
for filename in file_path_list:
    print('  -', filename)

current working directory:  /workspace/home

Provided data files:
  - /workspace/home/event_data/2018-11-10-events.csv
  - /workspace/home/event_data/2018-11-20-events.csv
  - /workspace/home/event_data/2018-11-09-events.csv
  - /workspace/home/event_data/2018-11-11-events.csv
  - /workspace/home/event_data/2018-11-07-events.csv
  - /workspace/home/event_data/2018-11-27-events.csv
  - /workspace/home/event_data/2018-11-21-events.csv
  - /workspace/home/event_data/2018-11-15-events.csv
  - /workspace/home/event_data/2018-11-17-events.csv
  - /workspace/home/event_data/2018-11-06-events.csv
  - /workspace/home/event_data/2018-11-24-events.csv
  - /workspace/home/event_data/2018-11-23-events.csv
  - /workspace/home/event_data/2018-11-04-events.csv
  - /workspace/home/event_data/2018-11-30-events.csv
  - /workspace/home/event_data/2018-11-29-events.csv
  - /workspace/home/event_data/2018-11-01-events.csv
  - /workspace/home/event_data/2018-11-16-events.csv
  - /workspace/home/event_data/20

#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [3]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:

# reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
 # extracting each data row one by one and append it        
        for line in csvreader:
            #print(line)
            full_data_rows_list.append(line) 
            
# uncomment the code below if you would like to get total number of rows 
print('Number of rows from files (in full_data_rows_list)',len(full_data_rows_list))
# uncomment the code below if you would like to check to see what the list of event data rows will look like
#print(full_data_rows_list)
#print(full_data_rows_list[34])

# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


Number of rows from files (in full_data_rows_list) 8056


In [4]:
# check the number of rows in your csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print('Number of data rows (now in event_datafile_new.csv)', sum(1 for line in f))

Number of data rows (now in event_datafile_new.csv) 6821


# Part II. Apache Cassandra coding  

## Working with the CSV file titled <font color=red>event_datafile_new.csv</font>, located within the Workspace directory.  The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

#### Creating a Cluster

In [5]:
# This should make a connection to a Cassandra instance your local machine 
# (127.0.0.1)

from cassandra.cluster import Cluster
cluster = Cluster()

# To establish connection and begin executing queries, need a session
session = cluster.connect()

#### Create Keyspace

In [6]:
SQL_CREATE_KEYSPACE = "CREATE KEYSPACE IF NOT EXISTS project_one_b_keyspace \
    WITH replication = { 'class': 'SimpleStrategy', 'replication_factor': 1 };" 

try:
    session.execute(SQL_CREATE_KEYSPACE)
    print('keyspace created')
except Exception as e:
    print(f"An error occurred creating the keyspace: {e}")

keyspace created


#### Set Keyspace

In [7]:
try:
    session.set_keyspace('project_one_b_keyspace')
    print('keyspace set')
except Exception as e:
    print(f"An error occurred setting the keyspace: {e}") 

keyspace set


## Query 1

### REQUIREMENT: 

#### artist, song title and song's length in the music app history that was heard during  session id = 338, and item in session  = 4 

### FIELDS: 

#### The fields in this table are the fields required for the required output (e.g. artist, song title and song's length) and the fields used for the filtering (e.g. session_id and item_in_session).  This table/query has no sorting requirements.  

### PRIMARY KEY:  

#### The primary key must be unique for each record in the table.  For this table/query, a unique record is an item in a session, so the primary key must include the fields session_id and item_in_session.

#### The filter criteria fields are session_id and item_in_session.  The primary key must include the filter criteria fields, though only one of the fields HAS to be in the partition key.  At least one of them must be the partition key.  In this case, we definitely want to have the session_id in the partition key, but the item_in_session could be part of the partition key or it could be a clustering column.  However, since we have no sorting requirements and we have many user sessions and the sessions can include so many songs, it makes sense to use them both to parition the data.  As a result the partition key part of the primary key is a composite of the session_id and item_in_session values. 

In [8]:
SQL_CREATE_SONG_DETAILS = 'CREATE TABLE IF NOT EXISTS song_details \
(session_id int, item_in_session int, artist_name text, song_title text, song_length float, \
PRIMARY KEY (session_id, item_in_session))'

try:
    session.execute(SQL_CREATE_SONG_DETAILS)
    print('Created table song_details')
except Exception as e:
    print(f"An error occurred creating the table song_details: {e}")         

Created table song_details


In [ ]:
DATA_FILE = 'event_datafile_new.csv'

SQL_INSERT_SONG_DETAILS = "INSERT INTO song_details \
(session_id, item_in_session, artist_name, song_title, song_length) \
values (%s,%s,%s,%s,%s)" 

with open(DATA_FILE, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader: 
        session.execute(SQL_INSERT_SONG_DETAILS, \
                        (int(line[8]), int(line[3]), line[0], line[9], float(line[5])))

print('Populated table song_details')

In [ ]:
SQL_QUERY_SONG_DETAILS = 'SELECT artist_name, song_title, song_length FROM song_details'

filter_sessionId = 338 
filter_itemInSession = 4

sql_query_song_details_with_filter = SQL_QUERY_SONG_DETAILS + \
f" where session_id = {filter_sessionId} and item_in_session = {filter_itemInSession}" 

try:
    rows = session.execute(sql_query_song_details_with_filter)  
    for row in rows:
        print(f"Artist: {row[0]}     Song: {row[1]}     Song Length: {row[2]}") 
    print('\n\nDone with query of song_details') 
except Exception as e:
    print(f"An error occurred query of the table song_details: {e}")

## Query 2  

### REQUIREMENT: 

#### name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182 

### FIELDS: 

#### The fields in this table are the fields required for the required output (name of artist, song, and user's first and last names), the fields used for the filtering (e.g. userid and sessionid), and the fields use for the sorting (e.g. itemInSession).

### PRIMARY KEY:  

#### The primary key must be unique for each record in the table.  For this table/query, a unique record is an item in a session of a user, so the primary key must include those three fields, i.e. it must include item in session, session id, and user id.

#### The filter criteria fields are user id and session id.  The primary key must include the filter criteria fields, though only one of the fields HAS to be in the partition key.  At least one of them must be the partition key.  In this case, because we have many users and users have may sessions, it makes sense to use them both to parition the data.  As a result the partition key part of the primary key is a composite of the user and session id values.

#### The sort criteria field is the item in session.  This part of the primary key is a clustering column to insure the sort order. 

In [ ]:
SQL_CREATE_TABLE_SONG_BY_USER_SESSION = 'CREATE TABLE IF NOT EXISTS song_by_user_session \
(user_id int, session_id int, item_in_session int, artist_name text, song_title text, \
firstName text, lastName text,  \
PRIMARY KEY ((user_id, session_id), item_in_session) )'

try: 
    session.execute(SQL_CREATE_TABLE_SONG_BY_USER_SESSION)
    print('created table song_by_user_session')
except Exception as e:
    print(f"An error occurred creating the table song_by_user_session: {e}") 

In [ ]:
SQL_INSERT_SONG_BY_USER_SESSION = "INSERT INTO song_by_user_session \
(user_id, session_id, item_in_session, artist_name, song_title, firstName, lastName) \
values (%s,%s,%s,%s,%s,%s,%s)" 

with open(DATA_FILE, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader: 
        session.execute(SQL_INSERT_SONG_BY_USER_SESSION, \
                        (int(line[10]), int(line[8]), int(line[3]), line[0], line[9], line[1], line[4]) )

print('Populated table song_by_user_session') 

In [ ]:
SQL_QUERY_SONG_BY_USER_SESSION = 'SELECT artist_name, song_title, firstName, lastName FROM song_by_user_session' 

filter_userid = 10
filter_itemInSession = 182

sql_query_with_filter_song_in_user_session = SQL_QUERY_SONG_BY_USER_SESSION + \
f" where user_id = {filter_userid} and session_id = {filter_itemInSession}" 

try:
    rows = session.execute(sql_query_with_filter_song_in_user_session) 
    
    for row in rows:
        print(f"Artist: {row[0]}     Song: {row[1]}     User: {row[2]} {row[3]}")

    print('\n\nDone with query of song_by_user_session') 
except Exception as e:
    print(f"An error occurred query of the table song_by_user_session: {e}")

## Query 3

### REQUIREMENT: 

#### user name (first and last) in music app history who listened to the song 'All Hands Against His Own'  

### FIELDS: 

#### The fields in this table are the fields required for the required output (e.g. user's first and last names), the fields used for the filtering (e.g. song), and the fields required to have a unique key for each row (e.g. UserIds).  We have to include the UserId field because two or more people could have the same name.  There is no sorting requirement for this query/table. 

### PRIMARY KEY:   

#### The primary key must be unique for each record in the table.  For this table/query, a unique record is a userId and song_title.  We cannot use first and last names as part of the unique partition key because because two or more people could have the same name.   So the primary key must include the two fields userId and song_title. 

#### The filter criteria field is song_title so it is the partition key.   

#### the userId makes the primary key unique but it is not a filter criteria so it is a clustered column.
 

In [ ]:
SQL_CREATE_TABLE_USER_BY_SONG = 'CREATE TABLE IF NOT EXISTS user_by_song \
(song_title text, user_id int, firstName text, lastName text, \
PRIMARY KEY (song_title, user_id))'

try: 
    session.execute(SQL_CREATE_TABLE_USER_BY_SONG)
    print('created table user_by_song')
except Exception as e:
    print(f"An error occurred creating the table user_by_song: {e}")                   

In [ ]:
SQL_INSERT_USER_BY_SONG = "INSERT INTO user_by_song (song_title, user_id, firstName, lastName) values (%s,%s,%s,%s)" 

with open(DATA_FILE, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader: 
        session.execute(SQL_INSERT_USER_BY_SONG, (line[9], int(line[10]), line[1], line[4]) )

print('Populated table user_by_song') 

In [ ]:
SQL_QUERY_USER_BY_SONG = 'SELECT firstName, lastName FROM user_by_song'

filter_song_title = 'All Hands Against His Own'

sql_query_song_user_with_filter = SQL_QUERY_USER_BY_SONG + f" where song_title = \'{filter_song_title}\'"  

try:
    rows = session.execute(sql_query_song_user_with_filter) 
     
    for row in rows:
        print(f"{row[0]} {row[1]}") 

    print('\n\nDone with query of user_by_song')
except Exception as e:
    print(f"An error occurred query of the table user_by_song: {e}")

## Cleanup

### Drop the tables before closing out the sessions

In [ ]:
try:
    session.execute('DROP TABLE song_details') 
    print('Dropped table song_details')
except Exception as e:
    print(f"An error occurred dropping the table song_details: {e}")

In [ ]:
try:
    session.execute('DROP TABLE song_by_user_session') 
    print('Dropped table song_by_user_session')
except Exception as e:
    print(f"An error occurred dropping the table song_by_user_session: {e}")

In [ ]:
try:
    session.execute('DROP TABLE user_by_song') 
    print('Dropped table user_by_song')
except Exception as e:
    print(f"An error occurred dropping the table user_by_song: {e}")

### Close the session and cluster connection¶

In [ ]:
session.shutdown()
cluster.shutdown()